# CityBikes

Send a request to CityBikes for the city of your choice. 

In [22]:
# Cell for declaring imports and constants used throughout this Notebook

import requests
import pandas as pd

VANCOUVER_CONSTS = {} # Empty dictionary containing constants
VANCOUVER_CONSTS['base_endpoint'] = 'http://api.citybik.es/v2/'

#### Step 1:  Call the CityBikes "networks" API to find the network_id for Vancouver
Per documentation on https://api.citybik.es/v2/#net_resource, the Endpoint is: http://api.citybik.es/v2/networks

I have already made the GET request in my browser and pasted the JSON output to codybeautify.org/jsonviewer to visualize the JSON structure, to make it easier to write the code to get the information into variables:

<img src='../images/citybikes_networks_payload_structure.png' style='width:70%;'>

In [23]:
# Step 1a: Make the REST GET request

endpoint = VANCOUVER_CONSTS['base_endpoint'] + 'networks'

header_dict = {
    'accept' : 'application/json'
}

response = requests.get(endpoint, headers=header_dict)
networks_dict = response.json()

# Parse the networks_dict to find the:
# 1) href for the CityBikes endpoint for Vancouver
# 2) name for the mobile bikes service for Vancouver
# 3) network_id for the mobile bieks service for Vancouver
# 4) latitutde for Vancouver (just in case we'll need it)
# 5) longitude for Vancouver (just in case we'll need it)

networks_array = networks_dict['networks']

for network in networks_array:
    if (network['location']['city'] == 'Vancouver'):
        VANCOUVER_CONSTS['latitude'] = network['location']['latitude']
        VANCOUVER_CONSTS['longitude'] = network['location']['longitude']
        VANCOUVER_CONSTS['network_name'] = network['name']
        VANCOUVER_CONSTS['network_id'] = network['id']
        VANCOUVER_CONSTS['network_endpoint'] = network['href']

In [24]:
# Show the VANCOUVER_CONSTS thus far:
for key, value in VANCOUVER_CONSTS.items():
    print(f"VANCOUVER_CONSTS[{key}]: {value}")

VANCOUVER_CONSTS[base_endpoint]: http://api.citybik.es/v2/
VANCOUVER_CONSTS[latitude]: 49.2827
VANCOUVER_CONSTS[longitude]: -123.1207
VANCOUVER_CONSTS[network_name]: Mobi
VANCOUVER_CONSTS[network_id]: mobibikes
VANCOUVER_CONSTS[network_endpoint]: /v2/networks/mobibikes


#### Step 2:  Call the CityBikes "network_id" API to find the bike stations for Vancouver
Per documentation on https://api.citybik.es/v2/#net_resource, the Endpoint is: http://api.citybik.es/v2/networks/<network_id>

Furthermore, we can limit the fields by passing the query parameter "fields=<fieldname1>,<fieldname2>,<fieldname3>"

I have already made the GET request in my browser to https://api.citybik.es/v2/networks/mobibikes?fields=id,name,stations pasted the JSON output to codybeautify.org/jsonviewer to visualize the JSON structure, to make it easier to write the code to get the information into variables:

<img src='../images/citybikes_mobibikes_payload_structure.png' style='width:70%;'>

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [33]:
# Step 2a: Make the REST GET request

fields_query_str = '?fields=id,name,stations'
endpoint = VANCOUVER_CONSTS['base_endpoint'] + 'networks' + '/' + VANCOUVER_CONSTS['network_id'] + fields_query_str
print(endpoint)

header_dict = {
    'accept' : 'application/json'
}

response = requests.get(endpoint, headers=header_dict)
stations_response_dict = response.json()

http://api.citybik.es/v2/networks/mobibikes?fields=id,name,stations


In [39]:
# Step 2b: Parse stations_dict

# Parse the stations_dict to find each station's:
# 1) id
# 2) name (city cross streets)
# 3) latitude
# 4) longitude
# 5) slots (taking this to mean the total number of bikes available to be rented).
#    Choosing to use this as a measure of number of bikes available at this station, and not "empty_slots", or "free_bikes", since those numbers can fluctuate depending on
#    The day, season and time of day (including the time at which I make my request!) and IMHO aren't a good measure of the network's belief in the potential of station location.

# Declare the dictionary that will hold each column in the eventual DataFrame
stations_dict = {
    'id' : [],
    'name' : [],
    'slots' : [],
    'lat' : [],
    'long' : []
}

print(stations_response_dict)

stations_array = stations_response_dict['network']['stations']

for station in stations_array:
        stations_dict['id']    .append( station['id'])
        stations_dict['name']  .append( station['name'])
        stations_dict['slots'] .append( station['extra']['slots'])
        stations_dict['lat']   .append( station['latitude'])
        stations_dict['long']  .append( station['longitude'])

{'network': {'company': ['Vanncouver Bike Share Inc.', 'CycleHop LLC', 'City of Vancouver', 'Shaw Communications Inc.', 'Fifteen'], 'gbfs_href': 'https://vancouver-gbfs.smoove.pro/gbfs/2/gbfs.json', 'href': '/v2/networks/mobibikes', 'id': 'mobibikes', 'location': {'city': 'Vancouver', 'country': 'CA', 'latitude': 49.2827, 'longitude': -123.1207}, 'name': 'Mobi', 'stations': [{'empty_slots': 17, 'extra': {'ebikes': 5, 'has_ebikes': True, 'last_updated': 1697737221, 'normal_bikes': 14, 'renting': True, 'returning': True, 'slots': 36, 'uid': '0001'}, 'free_bikes': 19, 'id': '7a19c49f486d7c0c02b3685d7b240448', 'latitude': 49.262487, 'longitude': -123.114397, 'name': '10th & Cambie', 'timestamp': '2023-10-19T17:42:19.349000Z'}, {'empty_slots': 12, 'extra': {'ebikes': 1, 'has_ebikes': True, 'last_updated': 1697737019, 'normal_bikes': 3, 'renting': True, 'returning': True, 'slots': 16, 'uid': '0004'}, 'free_bikes': 4, 'id': '32603a87cfca71d0f7dfa3513bad69d5', 'latitude': 49.274566, 'longitude

In [40]:
# Show the stations_dict:
for key, value in stations_dict.items():
    print(f"stations_dict[{key}]: {value}")

stations_dict[id]: ['7a19c49f486d7c0c02b3685d7b240448', '32603a87cfca71d0f7dfa3513bad69d5', '6d42fa40360f9a6b2bf641c7b8bb2862', '66f873d641d448bd1572ab086665a458', '485d4d24c803cfde829ab89699fed833', 'b07d513f87897cff7319a0e59d7e567c', 'd1c9556d92f329703229fb3e33899a3b', '95e624191c655f50e401d280cd39a9ad', '3e112f9fe877e0003780386359060ae6', '6993b3dbb0758927967592ea612a2b1e', 'bf8408067b0e0c963f3ff526977bcef3', 'd2244f1a81d317c76cabd65e81250835', '0b543fc4e694fe07a54dac48bb1b3390', 'fbb1d30d7f30b049873f5be5688563d4', '7181df50ee1d1a05559399211382b7c8', '2ef000bc9d4650d8cc6198c4d570b19f', 'b7683a86874e6300fd89f4cc0b075f57', '3367c1e1dbf73caf7c10e4c96a92f58d', '1d7e73b3321ebb6713774d376247d9f9', '664502c28e8147615f17b8ea355f40d0', '6c24a62b5ee0a2a8014eb0dfb6d40025', 'fad9ebcf614dd1a72593a34072ff76f8', '3a4131fa94946d93d3529136efd62413', '25318ec3e06fae1397a3a2f645104149', '444991edc9550be0908f032da5e3c95f', '434b6f4e7fc30198f901b60fc905f609', 'f97d5bf41f35aa181288429fb52e3488', 'fef69fb

Put your parsed results into a DataFrame.

In [58]:
df = pd.DataFrame(stations_dict)
print(f"df.dtypes:\n{df.dtypes}")
print("\n" + "df.info():")
print(df.info())
print(f"\ndf['id'].nunique() = {df['id'].nunique()}")
print(f"\ndf['slots'].value_counts() = {df['slots'].value_counts()}\n")
df.head()

df.dtypes:
id        object
name      object
slots      int64
lat      float64
long     float64
dtype: object

df.info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      245 non-null    object 
 1   name    245 non-null    object 
 2   slots   245 non-null    int64  
 3   lat     245 non-null    float64
 4   long    245 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 9.7+ KB
None

df['id'].nunique() = 245

df['slots'].value_counts() = slots
14    48
16    46
18    38
20    37
22    19
26    12
12    12
28     7
30     5
24     5
40     3
19     3
36     2
32     2
17     1
38     1
15     1
23     1
34     1
25     1
Name: count, dtype: int64



,id,name,slots,lat,long
0,7a19c49f486d7c0c02b3685d7b240448,10th & Cambie,36,49.262487,-123.114397
1,32603a87cfca71d0f7dfa3513bad69d5,Yaletown-Roundhouse Station,16,49.274566,-123.121817
2,6d42fa40360f9a6b2bf641c7b8bb2862,Dunsmuir & Beatty,26,49.279764,-123.110154
3,66f873d641d448bd1572ab086665a458,12th & Yukon (City Hall),16,49.260599,-123.113504
4,485d4d24c803cfde829ab89699fed833,8th & Ash,16,49.264215,-123.117772


In [59]:
# Save this dataframe to a .csv file so it can be retrieved in other notebooks from this project
df.to_csv('../data/citybikes_vancouver.csv', index=False)